In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
figure(num=None, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')
import seaborn as sns

In [ ]:
label_file_path = "../input/us_census_full/labels.txt"
with open(label_file_path) as f:
    names = [name.strip().replace(' ', '_') for name in f.readlines()]

In [ ]:
train_path = "../input/us_census_full/census_income_learn.csv"
test_path = "../input/us_census_full/census_income_test.csv"
train = pd.read_csv(train_path, names=names, sep=", ")
test = pd.read_csv(test_path, names=names, sep=", ")

In [ ]:
print("Train data size : " + str(train.shape))
print("Test data size : " + str(test.shape))

In [ ]:
train.sample(5)

In [ ]:
train.sample(5)

In [ ]:
def get_objects_ratios(df):
    for col in df.columns:
        dtype = str(df[col].dtype)
        if dtype == "object":
            ratio_count = pd.DataFrame(df[col].value_counts())
            ratio_count["ratio"] = ratio_count/ratio_count.sum()
            ratio_count.sort_values('ratio',ascending=False,inplace=True)
            display(ratio_count)

class_of_worker -> categorical<br/>
education -> ordinal?<br/>
enroll_in_edu_inst_last_wk -> categorical<br/>
marital_stat -> categorical<br/>
major_industry_code -> categorical
major_occupation_code -> categorical<br/>
race -> categorical<br/>
hispanic_origin -> categorical<br/>
sex -> categorical<br/>
member_of_a_labor_union -> categorical<br/>
reason_for_unemployment -> categorical<br/>
full_or_part_time_employment_stat -> categorical<br/>
tax_filer_status -> categorical<br/>
region_of_previous_residence -> categorical<br/>
state_of_previous_residence -> categorical, **Not in universe & ?**<br/>
detailed_household_and_family_stat -> categorical<br/>
migration_code-change_in_msa -> categorical, **Not in universe & Not identifiable & ?**<br/>
migration_code-change_in_reg -> categorical, **Not in universe & ?**<br/>
migration_code-move_within_reg -> categorical, **Not in universe & ?**<br/>
live_in_this_house_1_year_ago -> categorical<br/>
migration_prev_res_in_sunbelt -> categorical, **Not in universe & ?**<br/>
family_members_under_18 -> categorical<br/>
country_of_birth_father -> categorical<br/>
country_of_birth_mother -> categorical<br/>
country_of_birth_self -> categorical<br/>
citizenship -> categorical<br/>
fill_inc_questionnaire_for_veteran's_admin -> categorical<br/>
income -> categorical<br/>

In [ ]:
get_objects_ratios(train)

In [ ]:
get_objects_ratios(test)

In [ ]:
# replace 'Not in universe', 'Not identifiable' and '?' by null
train.replace("Not in universe", np.nan, inplace=True)
train.replace("Not identifiable", np.nan, inplace=True)
train.replace("?", np.nan, inplace=True)

In [ ]:
def missing_data(df):
    total = df.isnull().sum()
    percent = (df.isnull().sum()/df.shape[0]*100)
    missing_values = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in df.columns:
        dtype = str(df[col].dtype)
        types.append(dtype)
    missing_values['Types'] = types
    missing_values.sort_values('Total',ascending=False,inplace=True)
    return(np.transpose(missing_values))
missing_data(train)

In [ ]:
continuous_cols = [
    "age",
    "wage_per_hour",
    "capital_gains",
    "capital_losses",
    "dividends_from_stocks",
    "num_persons_worked_for_employer",
    "weeks_worked_in_year",
    "instance_weight"
]

In [ ]:
train[continuous_cols].hist(bins=50, figsize=(20,15))
plt.tight_layout(pad=0.4)
plt.show()